In [1]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
# 用於二元目標變數
from scipy.stats import pointbiserialr
from functools import reduce
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE

In [62]:
# Import GDP Data
from data import OverallGeneralEcon, df, MacroInfo, JuridicalInfo, Dream_report
from module1 import predict_for_month, find_best_threshold, predicted_labels, predicted_accuracy

##### MacroEcon
FX-Foriegn-exchange rate


CCI-台灣消費者信心指數

### 預計可使用的變數

In [3]:
# 名目、實質、時間
"""
券商預估、歷史營收增長
產業: 存貨量、產能利用率、訂單數量

MacroInfo
名目有效匯率指數–台灣
台灣消費者信心指數
台灣消費者信心指數–未來半年國內經濟景氣
台灣消費者信心指數–未來半年國內物價水準
美國消費者信心指數
經濟成長率(GDP)–全年
--
實質國內生產毛額(GDP)–台幣 -> 只更新到202009
實質GDP成長率(台灣) -> 只更新到202212
--
貨幣總額(M1A)日平均、貨幣總額(M2)日平均、貨幣總額年增率(M1A)日平均
臺灣製造業PMI - 新增出口訂單 - 基礎原物料
台灣外銷訂單金額–全球
消費者物價指數(CPI)、消費者物價指數(CPI)年增率、核心消費者物價指數
美國核心PPI、美國核心PPI月增率
進口物價指數、出口物價指數年增率

sysallunTWESPF
外銷訂單_美金計價(百萬美元)_合計_年增率(%)(月)
景氣同時指標_工業生產指數(Index 2006=100)_年增率(%)(月)

sysTaiStaValMAC
景氣指標統計_景氣領先指標不含趨勢指數(點)_年增率(%)(月)

其他
財報、營收
"""

'\n券商預估、歷史營收增長\n產業: 存貨量、產能利用率、訂單數量\n\nMacroInfo\n名目有效匯率指數–台灣\n台灣消費者信心指數\n台灣消費者信心指數–未來半年國內經濟景氣\n台灣消費者信心指數–未來半年國內物價水準\n美國消費者信心指數\n經濟成長率(GDP)–全年\n--\n實質國內生產毛額(GDP)–台幣 -> 只更新到202009\n實質GDP成長率(台灣) -> 只更新到202212\n--\n貨幣總額(M1A)日平均、貨幣總額(M2)日平均、貨幣總額年增率(M1A)日平均\n臺灣製造業PMI - 新增出口訂單 - 基礎原物料\n台灣外銷訂單金額–全球\n消費者物價指數(CPI)、消費者物價指數(CPI)年增率、核心消費者物價指數\n美國核心PPI、美國核心PPI月增率\n進口物價指數、出口物價指數年增率\n\nsysallunTWESPF\n外銷訂單_美金計價(百萬美元)_合計_年增率(%)(月)\n景氣同時指標_工業生產指數(Index 2006=100)_年增率(%)(月)\n\nsysTaiStaValMAC\n景氣指標統計_景氣領先指標不含趨勢指數(點)_年增率(%)(月)\n\n其他\n財報、營收\n'

### *【訓練步驟：資料處理 → 分割 train、test dataset → model fit  → 計算出機率&閾值 → 計算準確率】

### 總經資料處理

In [4]:
# Improt data
# 名目有效匯率指數
NominalExchageRate = MacroInfo[MacroInfo['名稱']=='名目有效匯率指數–台灣'][['年月','數值','名稱']].rename(columns={'年月':'ddate', '數值':'value', '名稱':'category'})
display(NominalExchageRate.head(3))

# 台灣消費者信心指數
TW_ICS = MacroInfo[MacroInfo['名稱']=='台灣消費者信心指數'][['年月','數值','名稱']].rename(columns={'年月':'ddate', '數值':'value', '名稱':'category'})
display(TW_ICS.tail(3))

# 消費者物價指數(CPI)年增率
TW_CPIGrowth = MacroInfo[MacroInfo['名稱']=='消費者物價指數(CPI)年增率'][['年月','數值','名稱']].rename(columns={'年月':'ddate', '數值':'value', '名稱':'category'})
display(TW_CPIGrowth.head(3))

# 經濟成長率(GDP)–全年
TW_GDPGrowth = MacroInfo[MacroInfo['名稱']=='經濟成長率(GDP)–單季'][['年月','數值','名稱']].rename(columns={'年月':'ddate', '數值':'value', '名稱':'category'})
display(TW_GDPGrowth.head(3))

,ddate,value,category
916,202312,101.78,名目有效匯率指數–台灣
2069,202311,101.05,名目有效匯率指數–台灣
3315,202310,100.99,名目有效匯率指數–台灣


,ddate,value,category
258880,201403,80.96,台灣消費者信心指數
261195,201402,82.93,台灣消費者信心指數
263490,201401,80.83,台灣消費者信心指數


,ddate,value,category
316,202312,2.71,消費者物價指數(CPI)年增率
1357,202311,2.90,消費者物價指數(CPI)年增率
2575,202310,3.05,消費者物價指數(CPI)年增率


,ddate,value,category
0,202412,1.06,經濟成長率(GDP)–單季
27,202409,2.12,經濟成長率(GDP)–單季
42,202406,4.77,經濟成長率(GDP)–單季


In [5]:
"""
# 合計買賣超金額(百萬) = 上市櫃合計買進金額(百萬)-上市櫃合計賣出金額(百萬)、興櫃買賣超金額(百萬) 
TradingExceeds2 = JuridicalInfo2[['日期', '名稱','合計買賣超金額(百萬)','興櫃買賣超金額(百萬)']]
TradingExceeds2 = TradingExceeds2.fillna(0)

# Create English Column names
TradingExceeds2 = TradingExceeds2.rename({'日期':'ddate', '名稱':'classification', '合計買賣超金額(百萬)':'Trans_amount(mln)', '興櫃買賣超金額(百萬)':'Emerge_amount(mln)'}, axis='columns') 
#TradingExceeds2.head(10)
"""

"\n# 合計買賣超金額(百萬) = 上市櫃合計買進金額(百萬)-上市櫃合計賣出金額(百萬)、興櫃買賣超金額(百萬) \nTradingExceeds2 = JuridicalInfo2[['日期', '名稱','合計買賣超金額(百萬)','興櫃買賣超金額(百萬)']]\nTradingExceeds2 = TradingExceeds2.fillna(0)\n\n# Create English Column names\nTradingExceeds2 = TradingExceeds2.rename({'日期':'ddate', '名稱':'classification', '合計買賣超金額(百萬)':'Trans_amount(mln)', '興櫃買賣超金額(百萬)':'Emerge_amount(mln)'}, axis='columns') \n#TradingExceeds2.head(10)\n"

### 三大法人買賣超資料處理

In [50]:
# 上市櫃公司三大法人買賣超、持股比率增減(分個股)
JuridicalInfo = JuridicalInfo.rename({'日期':'ddate', '股票代號':'stockid', '三大法人月初以來買賣超金額(千)':'GrandTransAmount(K)', '三大法人持股比率增減':'ShareRatio_change', '三大法人近1日買賣超金額(千)':'DailyTransAmount(K)'}, axis=1)
#display(JuridicalInfo.head())

,ddate,stockid,GrandTransAmount(K),ShareRatio_change,DailyTransAmount(K)
0,20240124,1101,-1359231.0,-0.01,-70978.0
1,20240124,1101B,NaN,0.00,NaN
2,20240124,1102,-556926.0,-0.01,-21277.0
3,20240124,1103,-10120.0,0.00,-397.0
4,20240124,1104,-22396.0,0.01,1907.0


In [52]:
JuridicalInfo['date'] = pd.to_datetime(JuridicalInfo['ddate']) # make sure the date column is datetime type
JuridicalInfo['last_date_of_month'] = JuridicalInfo['date'] + pd.offsets.MonthEnd(1)  # get the last day of the month
display(JuridicalInfo)


,ddate,stockid,GrandTransAmount(K),ShareRatio_change,DailyTransAmount(K),date,last_date_of_month
0,20240124,1101,-1359231.0,-0.01,-70978.0,2024-01-24,2024-01-31
1,20240124,1101B,NaN,0.00,NaN,2024-01-24,2024-01-31
2,20240124,1102,-556926.0,-0.01,-21277.0,2024-01-24,2024-01-31
3,20240124,1103,-10120.0,0.00,-397.0,2024-01-24,2024-01-31
4,20240124,1104,-22396.0,0.01,1907.0,2024-01-24,2024-01-31
...,...,...,...,...,...,...,...
6809779,20060102,9949,0.0,0.00,0.0,2006-01-02,2006-01-31
6809780,20060102,9950,0.0,0.00,0.0,2006-01-02,2006-01-31
6809781,20060102,9951,0.0,0.00,0.0,2006-01-02,2006-01-31
6809782,20060102,9955,0.0,0.00,0.0,2006-01-02,2006-01-31


In [8]:
TW_NominalGDP = OverallGeneralEcon[['time','Nominal_GDP']]
TW_NominalGDP = TW_NominalGDP.rename(columns={'time':'ddate'})
#print(TW_NominalGDP.head(3))
TW_NominalGDP['ddate'] = TW_NominalGDP['ddate'].astype(str)

In [9]:
# 將GDP季資料先合併
GDP_quarterly = pd.merge(TW_NominalGDP, TW_GDPGrowth, on='ddate', how='left')
GDP_quarterly.columns = ['ddate', 'NominalGDP', 'GDP_growth', 'category_GDPgrowth']
#GDP_quarterly.head(3)

In [10]:
# (月)總經資料
dfs = [NominalExchageRate, TW_ICS, TW_CPIGrowth]
MergedMacroData = reduce(lambda left, right: pd.merge(left, right, on='ddate', how='left'), dfs)

# Rename columns
MergedMacroData.columns = ['ddate', 'FX', 'category_FX', 'CCI',
                          'category_CCI', 'CPI_growth', 'category_CPIgrowth']

#display(MergedMacroData)
# FX-Foriegn-exchange rate
# CCI-台灣消費者信心指數

In [11]:
MergedMacroData = pd.merge(MergedMacroData, GDP_quarterly, on='ddate', how='left')

In [12]:
MergedMacroData['NominalGDP'] = MergedMacroData['NominalGDP'].ffill()
MergedMacroData['GDP_growth'] = MergedMacroData['GDP_growth'].ffill()
MergedMacroData['category_GDPgrowth'] = MergedMacroData['category_GDPgrowth'].ffill()

In [13]:
MergedMacroData.head(9)

,ddate,FX,category_FX,CCI,category_CCI,CPI_growth,category_CPIgrowth,NominalGDP,GDP_growth,category_GDPgrowth
0,202312,101.78,名目有效匯率指數–台灣,70.06,台灣消費者信心指數,2.71,消費者物價指數(CPI)年增率,NaN,NaN,NaN
1,202311,101.05,名目有效匯率指數–台灣,69.40,台灣消費者信心指數,2.90,消費者物價指數(CPI)年增率,NaN,NaN,NaN
2,202310,100.99,名目有效匯率指數–台灣,69.06,台灣消費者信心指數,3.05,消費者物價指數(CPI)年增率,NaN,NaN,NaN
3,202309,101.07,名目有效匯率指數–台灣,66.73,台灣消費者信心指數,2.93,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季
4,202308,100.67,名目有效匯率指數–台灣,67.51,台灣消費者信心指數,2.53,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季
5,202307,101.31,名目有效匯率指數–台灣,68.39,台灣消費者信心指數,1.88,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季
6,202306,102.80,名目有效匯率指數–台灣,66.66,台灣消費者信心指數,1.75,消費者物價指數(CPI)年增率,5697026.0,1.36,經濟成長率(GDP)–單季
7,202305,101.84,名目有效匯率指數–台灣,63.67,台灣消費者信心指數,2.02,消費者物價指數(CPI)年增率,5697026.0,1.36,經濟成長率(GDP)–單季
8,202304,101.35,名目有效匯率指數–台灣,63.18,台灣消費者信心指數,2.35,消費者物價指數(CPI)年增率,5697026.0,1.36,經濟成長率(GDP)–單季


### 單純以總經變數訓練模型，準確率約51% -TheWhole

In [14]:
# Rename the column
df = df.rename(columns={'direction':'real_direction'})

In [15]:
# 合併df以及總經資料
TheWhole = pd.merge(df[['ddate', 'stockid', 'real_direction']].astype({'ddate':'str'}), 
                    MergedMacroData, on='ddate', how='left')
#TheWhole[TheWhole['ddate']=='202302'].head(3)

In [16]:
#TheWhole.columns
TheWhole = TheWhole.dropna()
#TheWhole.head()

In [17]:
# Split data by TimeSeriesSplit
tscv_macro = TimeSeriesSplit(n_splits=50)

# each split contains train set & test set 'index'
splits_macro = list(tscv_macro.split(TheWhole))

In [18]:
# 使用最後一組的 trained model index 切分 training & testing data
train_idx_macro = splits_macro[-1][0]
test_idx_macro = splits_macro[-1][1]

In [19]:
TargetCol = ['real_direction']
VarialbleCol = ['FX', 'CCI', 'CPI_growth',
                'NominalGDP', 'GDP_growth']

# Prepare data for separating
y_macro = TheWhole[TargetCol]
X_macro = TheWhole[VarialbleCol]

In [20]:
# Extract test and train data from overall data
X_train_macro, X_test_macro = X_macro.iloc[train_idx_macro], X_macro.iloc[test_idx_macro]
y_train_macro, y_test_macro = y_macro.iloc[train_idx_macro], y_macro.iloc[test_idx_macro]

In [21]:
# Train model based on the training data
model_macro = LogisticRegression()
model_fit_macro = model_macro.fit(X_train_macro, y_train_macro)

c:\Users\user1\anaconda3\envs\rev_pred_env\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [22]:
# Make predictions on the test data
# Calculate thresholds to determine the predicted results
# probability
# display the results(0/1)

In [23]:
# 預測結果為1的機率
prob_macro = predict_for_month(model_fit_macro, TheWhole, VarialbleCol)
threshold_macro = find_best_threshold(TheWhole['real_direction'], prob_macro)
#print(round(threshold_macro, 2)) #0.51

print(round(threshold_macro, 4))

0.5102


In [24]:
# 以閾值為界判斷預測值應為0或1
pred_macro = predicted_labels(prob_macro, threshold_macro)

In [25]:
# 計算總經數值正確性
predicted_accuracy(TheWhole['real_direction'], pred_macro) #0.51

0.51

### 總經指數+營收 訓練+預測 -TheWhole_rev

In [26]:
TheWhole['ddate'] = TheWhole['ddate'].astype(str)
df['ddate'] = df['ddate'].astype(str)

In [27]:
df = df.rename({'direction':'real_direction'}, axis='columns')
TheWhole_rev = pd.merge(df, TheWhole, on=('ddate', 'stockid', 'real_direction'), how='inner')
TheWhole_rev.head(5)

,ddate,stockid,real_direction,rev,mom,yoy,revd1,yoyd1,mond1,revd2,...,mond12,FX,category_FX,CCI,category_CCI,CPI_growth,category_CPIgrowth,NominalGDP,GDP_growth,category_GDPgrowth
0,202309,1101,0,8735157.0,-6.87,-16.05,9379408.0,-12.26,4.76,8953462.0,...,-2.67,101.07,名目有效匯率指數–台灣,66.73,台灣消費者信心指數,2.93,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季
1,202309,1102,1,6691905.0,3.43,-4.32,6469836.0,-8.48,-4.72,6790125.0,...,-1.06,101.07,名目有效匯率指數–台灣,66.73,台灣消費者信心指數,2.93,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季
2,202309,1103,0,232979.0,-2.49,13.50,238917.0,31.01,-10.39,266631.0,...,12.55,101.07,名目有效匯率指數–台灣,66.73,台灣消費者信心指數,2.93,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季
3,202309,1104,1,618592.0,0.23,3.59,617197.0,1.93,-3.15,637246.0,...,-1.38,101.07,名目有效匯率指數–台灣,66.73,台灣消費者信心指數,2.93,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季
4,202309,1108,0,404171.0,-6.10,16.60,430420.0,10.35,-2.43,441143.0,...,-11.13,101.07,名目有效匯率指數–台灣,66.73,台灣消費者信心指數,2.93,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季


In [28]:
Varialble_MacroRev = ['revd1', 'yoyd1',
                         'mond1', 'revd2', 'yoyd2', 'mond2', 'revd3', 'yoyd3', 'mond3', 'revd4',
                         'yoyd4', 'mond4', 'revd5', 'yoyd5', 'mond5', 'revd6', 'yoyd6', 'mond6',
                         'revd7', 'yoyd7', 'mond7', 'revd8', 'yoyd8', 'mond8', 'revd9', 'yoyd9',
                         'mond9', 'revd10', 'yoyd10', 'mond10', 'revd11', 'yoyd11', 'mond11',
                         'revd12', 'yoyd12', 'mond12', 'FX', 'CCI', 'CPI_growth',
                         'NominalGDP', 'GDP_growth']

Target_MacroRev = ['real_direction']

In [29]:
# Prepare data for separating
y_MacroRev = TheWhole_rev[Target_MacroRev]
X_MacroRev = TheWhole_rev[Varialble_MacroRev]

# Extract test and train data from overall data
X_train_MacroRev, X_test_MacroRev = X_MacroRev.iloc[train_idx_macro], X_MacroRev.iloc[test_idx_macro]
y_train_MacroRev, y_test_MacroRev = y_MacroRev.iloc[train_idx_macro], y_MacroRev.iloc[test_idx_macro]

In [30]:
# Train model based on the training data
model_MacroRev = LogisticRegression()
model_fit_MacroRev = model_MacroRev.fit(X_train_MacroRev, y_train_MacroRev)

c:\Users\user1\anaconda3\envs\rev_pred_env\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\user1\anaconda3\envs\rev_pred_env\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [31]:
# 預測結果為1的機率
prob_MacroRev = predict_for_month(model_fit_MacroRev, TheWhole_rev, Varialble_MacroRev)
threshold_MacroRev = find_best_threshold(TheWhole_rev['real_direction'], prob_MacroRev) # 0.5092
print(threshold_MacroRev)
#print(round(threshold_MacroRev, 4))

# 以閾值為界判斷預測值應為0或1
pred_MacroRev = predicted_labels(prob_MacroRev, threshold_MacroRev)

0.5091613123764465


In [32]:
# 計算總經數值正確性
predicted_accuracy(TheWhole_rev['real_direction'], pred_MacroRev) # 0.62

0.62

In [33]:
# 分月份


### 以MLE判斷模型及變數是否有效

In [34]:
# 針對總經+12月營收判斷
# 使用 RFE 進行變數選擇
selector = RFE(model_MacroRev, n_features_to_select=20) #留下20 個 features
selector = selector.fit(X_MacroRev, y_MacroRev)

# Get the ranking of each feature
featre_ranking = selector.ranking_

featre_ranking
# 輸出選擇的特徵
selected_features = X_MacroRev.columns[selector.support_]
print(selected_features)

c:\Users\user1\anaconda3\envs\rev_pred_env\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\user1\anaconda3\envs\rev_pred_env\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\user1\anaconda3\envs\rev_pred_env\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please 

Index(['revd1', 'yoyd1', 'mond1', 'yoyd2', 'yoyd3', 'mond3', 'yoyd4', 'mond4',
       'yoyd5', 'yoyd6', 'mond6', 'yoyd7', 'mond7', 'yoyd8', 'yoyd9', 'yoyd10',
       'mond11', 'revd12', 'yoyd12', 'mond12'],
      dtype='object')


In [35]:
selected_features = np.where(featre_ranking == 1)[0]

In [36]:
# 標準化
scaler = StandardScaler()
X_train_MacroRev = scaler.fit_transform(X_train_MacroRev)
X_test_MacroRev = scaler.transform(X_test_MacroRev)

model = LogisticRegression()
model.fit(X_train_MacroRev, y_train_MacroRev)

coeffients = model.coef_[0]
feature_importance = pd.DataFrame({'Feature': Varialble_MacroRev, 'Importance': np.abs(coeffients)})
feature_importance = feature_importance.sort_values('Importance', ascending=False)
#feature_importance.plot(x='Feature', y='Importance', kind='barh', figsize=(10, 6))
display(feature_importance[:20])

c:\Users\user1\anaconda3\envs\rev_pred_env\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\user1\anaconda3\envs\rev_pred_env\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,Feature,Importance
0,revd1,2.141860
33,revd12,1.481950
3,revd2,0.445347
15,revd6,0.383429
6,revd3,0.243646
18,revd7,0.217388
30,revd11,0.213559
12,revd5,0.156035
27,revd10,0.145844
2,mond1,0.075035


### 以重要度最高的20個變數訓練模型

In [37]:
#(feature_importance['Feature'][:20][0])
Varialble_import20 = list(feature_importance['Feature'][:20].astype(str))

In [38]:
Varialble_import20

['revd1',
 'revd12',
 'revd2',
 'revd6',
 'revd3',
 'revd7',
 'revd11',
 'revd5',
 'revd10',
 'mond1',
 'revd4',
 'CPI_growth',
 'CCI',
 'GDP_growth',
 'revd9',
 'yoyd1',
 'revd8',
 'NominalGDP',
 'FX',
 'mond7']

In [39]:
# Prepare data for separating
y_import20 = TheWhole_rev[Target_MacroRev]
X_import20 = TheWhole_rev[Varialble_import20]

# Separate data into test and train set
X_train_import20, X_test_import20 = X_import20.iloc[train_idx_macro], X_import20.iloc[test_idx_macro]
y_train_import20, y_test_import20 = y_import20.iloc[train_idx_macro], y_import20.iloc[test_idx_macro]

# Train model based on the training data
model_import20 = LogisticRegression()
model_fit_import20 = model_import20.fit(X_train_import20, y_train_import20)

c:\Users\user1\anaconda3\envs\rev_pred_env\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [40]:
# 預測結果為1的機率
prob_import20 = predict_for_month(model_fit_import20, X_import20, Varialble_import20)
threshold_import20 = find_best_threshold(y_import20, prob_import20) # 0.5092
#print(threshold_import20)
#print(round(threshold_import20, 4)) 

# 以閾值為界判斷預測值應為0或1
pred_import20 = predicted_labels(prob_import20, threshold_import20)
#print(pred_import20)

# 計算總經數值正確性
predicted_accuracy(y_import20, pred_import20) # 0.62

0.62

###### Conclusion: because the threshold is almost the same as the unselect variable ones, the accuracy is the same.

### 以前20重要的變數+三大法人+其他總經變數

In [65]:
TheWhole_rev.head()

,ddate,stockid,real_direction,rev,mom,yoy,revd1,yoyd1,mond1,revd2,...,mond12,FX,category_FX,CCI,category_CCI,CPI_growth,category_CPIgrowth,NominalGDP,GDP_growth,category_GDPgrowth
0,202309,1101,0,8735157.0,-6.87,-16.05,9379408.0,-12.26,4.76,8953462.0,...,-2.67,101.07,名目有效匯率指數–台灣,66.73,台灣消費者信心指數,2.93,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季
1,202309,1102,1,6691905.0,3.43,-4.32,6469836.0,-8.48,-4.72,6790125.0,...,-1.06,101.07,名目有效匯率指數–台灣,66.73,台灣消費者信心指數,2.93,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季
2,202309,1103,0,232979.0,-2.49,13.50,238917.0,31.01,-10.39,266631.0,...,12.55,101.07,名目有效匯率指數–台灣,66.73,台灣消費者信心指數,2.93,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季
3,202309,1104,1,618592.0,0.23,3.59,617197.0,1.93,-3.15,637246.0,...,-1.38,101.07,名目有效匯率指數–台灣,66.73,台灣消費者信心指數,2.93,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季
4,202309,1108,0,404171.0,-6.10,16.60,430420.0,10.35,-2.43,441143.0,...,-11.13,101.07,名目有效匯率指數–台灣,66.73,台灣消費者信心指數,2.93,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季


In [83]:
Dream_report['ddate'] = Dream_report['ddate'].astype(str)
Dream_report['stockid'] = Dream_report['stockid'].astype(int)
#Dream_report['ActualDirect']
type(Dream_report['ddate'][0])
type(Dream_report['stockid'][0])
Dream_report.loc[Dream_report['ddate'] == '202309']

,ddate,stockid,direction,ActualDirect
32378,202309,1101,1.0,0.0
32379,202309,1102,1.0,1.0
32380,202309,1103,1.0,0.0
32381,202309,1104,1.0,1.0
32382,202309,1108,1.0,0.0
...,...,...,...,...
34165,202309,9951,1.0,1.0
34166,202309,9955,0.0,1.0
34167,202309,9958,1.0,1.0
34168,202309,9960,0.0,0.0


In [91]:
len(Dream_report)

187966

In [76]:
type(TheWhole_rev['ddate'][0])
type(TheWhole_rev['stockid'][0])

numpy.int64

In [93]:
# Check for duplicate rows in TheWhole_rev
duplicates = TheWhole_rev.duplicated(subset=['ddate', 'stockid'], keep=False)
print("Number of duplicate rows in TheWhole_rev:", duplicates.sum())

Number of duplicate rows in TheWhole_rev: 0


True

In [87]:
if 'ActualDirect' in TheWhole_rev:
    print("Merge successful: 'ActualDirect' is in the merged dataframe.")
else:
    print("Merge unsuccessful: 'ActualDirect' is NOT in the merged dataframe.")

Merge successful: 'ActualDirect' is in the merged dataframe.
